# Efficient generation of dynamic pulses - Ramsey
This script implement a simulated Ramsey sequence, as described in the relative [blog post](https://blogs.zhinst.com/andrea/2021/09/08/efficient-generation-of-dynamic-pulses/)


Copyright (C) 2021 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

In [ ]:
#Define the parameters of the instrument
dataserver_host = 'localhost'    #Hostname or IP address of the dataserer
dev_hd = 'devNNNN'               #Device ID of the HDAWG
awg_core = 0                     #AWG Core (equal to output_channel//2)

In [ ]:
#Imports and initializations
from zhinst.ziPython import ziDAQServer
daq = ziDAQServer(dataserver_host, 8004, 6)

from hdawg_utils import HDAWG_Core
hdawg = HDAWG_Core(daq, dev_hd, awg_core)

## Ramsey sequence, time sweep

In [ ]:
#Waveform paramaters
hdawg.constants.PULSE_WIDTH = 10e-9 #ns
hdawg.constants.SIGMAS_PULSE = 6

#Sequence parameters
hdawg.constants.T_START = 16
hdawg.constants.T_END = 10240
hdawg.constants.T_STEP = 27

#Readout parameters
hdawg.constants.TRIGGER_LEN = 32
hdawg.constants.READOUT_LEN = 1024

#Creation of the command table
ct = []
for i in range(16):
    entry = {
        "index": i,
        "waveform": {
            "index": i
        }
    }
    ct.append(entry)

seq_definition = """
//Calculations
const PULSE_WIDTH_SAMPLE = PULSE_WIDTH*DEVICE_SAMPLE_RATE;
const PULSE_TOTAL_LEN = ceil(PULSE_WIDTH_SAMPLE*SIGMAS_PULSE*2/16)*16;

//Waveform definition
wave pulse = gauss(PULSE_TOTAL_LEN, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);

//Create shifted waveforms
cvar j;
for (j = 0; j < 16; j++) {
  wave w_shifted = join(zeros(j), pulse, zeros(16-j));  //Create the j-samples shifted waveform
  assignWaveIndex(1,w_shifted, j);                      //Assign index j to the waveform
}


//Execution, no averaging
var t = T_START;
var t_fine, coarse;
do {
    t_fine = t & 0xF;                    //The fine shift is the four least significant bits
    coarse = t & -0x10;                  //Check if coarse delay is needed (boolean). Equivalent to (t >= 16)

    resetOscPhase();                     //Reset the oscillator

    playWave(1,pulse);                   //Play first pulse, no shift
    if(coarse)
      playZero(t);                       //Evolution time t (coarse)
    executeTableEntry(t_fine);           //Play second pulse, fine shift
    
    playWave(marker(TRIGGER_LEN, 1));    //Readout trigger
    playZero(READOUT_LEN);               //Readout time

    t += T_STEP;                         //Increase wait time
} while (t < T_END);                     //Loop until the end
"""

hdawg.config(seq_definition, ct=ct)

In [ ]:
hdawg.run()

## Ramsey sequence, two-axis control

In [ ]:
#Waveform paramaters
hdawg.constants.PULSE_WIDTH = 10e-9 #ns
hdawg.constants.SIGMAS_PULSE = 6
hdawg.constants.FIRST_PULSE_INDEX = 0
hdawg.constants.SECOND_PULSE_INDEX = 1

#time parameters
hdawg.constants.TIME_SHIFT = 1234

#Sequence parameters
hdawg.constants.NUM_PHASES = 512
hdawg.constants.FIRST_PI_2 = 1023

#Readout parameters
hdawg.constants.TRIGGER_LEN = 32
hdawg.constants.READOUT_LEN = 1024

#Creation of the command table
ct = []
for i in range(hdawg.constants.NUM_PHASES):
    entry = {
        "index": i,
        "waveform": {
            "index": hdawg.constants.SECOND_PULSE_INDEX
        },
        "phase0": {
            "value": 360.0*i/hdawg.constants.NUM_PHASES,
            "increment": False
        }
    }
    ct.append(entry)

entry = {
    "index": hdawg.constants.FIRST_PI_2,
    "waveform": {
        "index": hdawg.constants.FIRST_PULSE_INDEX
    },
    "phase0": {
        "value": 0.0,
        "increment": False
    }
}
ct.append(entry)

seq_definition = """
//Calculations
const PULSE_WIDTH_SAMPLE = PULSE_WIDTH*DEVICE_SAMPLE_RATE;
const PULSE_TOTAL_LEN = ceil(PULSE_WIDTH_SAMPLE*SIGMAS_PULSE*2/16)*16;

const T_FINE = TIME_SHIFT & 0xF;
const T_COARSE = TIME_SHIFT & 0x7FFFFFF0;

//Waveform definition
wave pulse = gauss(PULSE_TOTAL_LEN, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);

//Waveform definition
wave pulse_1 = gauss(PULSE_TOTAL_LEN, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);
wave pulse_2 = join(zeros(T_FINE), pulse_1, zeros(16-T_FINE));
assignWaveIndex(1, pulse_1, FIRST_PULSE_INDEX);
assignWaveIndex(1, pulse_2, SECOND_PULSE_INDEX);


//Execution, no averaging
var phase = 0;
do {
    resetOscPhase();                     //Reset the oscillator

    executeTableEntry(FIRST_PI_2);       //Play first pulse, no shift, zero phase
    playZero(T_COARSE);                  //Evolution time t (coarse)
    executeTableEntry(phase);            //Play second pulse, fixed fine shift and added phase
    
    playWave(marker(TRIGGER_LEN, 1));    //Readout trigger
    playZero(READOUT_LEN);               //Readout time

    phase += 1;                          //Increase the phase by 2PI/NUM_PHASES
} while (phase < NUM_PHASES);            //Loop until the end
"""

hdawg.config(seq_definition, ct=ct)
hdawg.modulation("sine00")
hdawg.oscControl(True)

In [ ]:
hdawg.run()